# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp app

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
except: 
    from load_data import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import json
from git import Repo
import pandas as pd
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, html, Patch  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import random

## Load Data

In [ ]:
#| export
sol_geo = SolomonGeo.load_pickle("/testData/")
geo_df = sol_geo.geo_df

## Setup Dash Components

In [ ]:
#| export
# Build your components
# FYI the best themes seem to be: [Darkly, Flatly, Minty, Slate]
app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
server = app.server


mytitle = dcc.Markdown(children='')
mygraph = dcc.Graph(figure={})
geos = geo_df.loc[:, 'agg'].unique()
cen_vars = sol_geo.census_vars
dropdown_geo = dcc.Dropdown(options=geos,
                        value=geos[0],  # initial value displayed when page first loads
                        clearable=False)
dropdown_var = dcc.Dropdown(options=cen_vars,
                        value=cen_vars[-1],  # initial value displayed when page first loads
                        clearable=False)

## Create a sidebar
The sidebar will 

In [ ]:
#| export
# Note, for now I am not using a sidebar style as I do not want to fix the width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "24rem",
    "padding": "2rem 1rem",
    #"background-color": "#f8f9fa",
}


sidebar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        dbc.Nav(
            [
                html.P("Geography"), # TODO add a little info button here with link to geo explanation
                dropdown_geo,
                html.Br(),
                html.P("Data"), # TODO add a little info button here with link to geo explanation
                dropdown_var,
                #html.Br(),
                #dcc.Dropdown(id = 'three')

            ],
            vertical=True,
            pills=True,
        ),
    ],
    #style=SIDEBAR_STYLE,
)



## Customize the Layout

In [ ]:
#| export
app.layout = dbc.Container([
                dbc.Row([
                    dbc.Col(),

                    dbc.Col([mytitle],width = 9, style = {'margin-left':'7px','margin-top':'7px'})
                    ]),
                dbc.Row(
                    [dbc.Col(sidebar, width = 3),
                    dbc.Col([mygraph], width = 9)#, style = {'margin-left':'15px', 'margin-top':'7px', 'margin-right':'15px'})
                     ], justify = 'center'),                    
                ], fluid = True)

## Build callbacks to create interaction

#| export
# Callback allows components to interact
@app.callback(
    Output(mygraph, 'figure'),
    Output(mytitle, 'children'),
    Input(dropdown_geo, 'value')
)
def update_graph(user_input):  # function arguments come from the component property of the Input
    repo = Repo('.', search_parent_directories=True)
    fig = go.Figure(go.Choroplethmapbox(
        # TODO - may need to use old version of mapping to get visibility hack to work..
                            geojson=sol_geo.get_geojson(agg_filter = user_input),
                           locations=sol_geo.get_df(agg_filter = user_input).index,
                           z = sol_geo.get_df(agg_filter = user_input)['total_pop'],
                           colorscale="deep",
                            marker_line_width = 0,
                            zauto=True))

    fig.update_layout(mapbox_style = 'carto-positron',
                        mapbox_zoom = 5,
                        mapbox_center={"lat": -9.565766, "lon": 162.012453},
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    # returned objects are assigned to the component property of the Output
    return fig, '# Solomon Islands Data map - ' + user_input


In [ ]:
#| export
# Callback allows components to interact
@app.callback(
    Output(mygraph, 'figure'),
    Output(mytitle, 'children'),
    Input(dropdown_geo, 'value')
)
def update_graph(user_input):  # function arguments come from the component property of the Input
    repo = Repo('.', search_parent_directories=True)
    fig = px.choropleth_mapbox(sol_geo.get_df(agg_filter = user_input),
        # TODO - may need to use old version of mapping to get visibility hack to work..
                        #geojson = str(repo.working_tree_dir) + "/assets/sol_geo.json",
                            geojson=sol_geo.get_geojson(agg_filter = user_input),
                           locations=sol_geo.get_df(agg_filter = user_input).index,
                               color = 'total_pop',
                           color_continuous_scale="deep",
                               mapbox_style = 'carto-positron',
                               opacity = 0.8,
                        zoom = 5,
                        center={"lat": -9.565766, "lon": 162.012453}

    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    # returned objects are assigned to the component property of the Output
    return fig, '# Solomon Islands Data map - ' + user_input

### Update the colour columns based on data selection

In [ ]:
patched_figure = Patch()
print(patched_figure)


<write-only dash.Patch object at []>


In [ ]:
# cols_dd dictates the aggregation that will be visable
cols_dd = geo_df.loc[:, 'agg'].unique()
# we need to add this to select which trace 
# is going to be visible
visible = np.array(cols_dd)
# define traces and buttons at once
traces = []
buttons = []
# TODO if fails remember I changed visible from cols_dd
for value in cols_dd:
    traces.append(go.Choroplethmapbox(
                            geojson=sol_geo.get_geojson(agg_filter = value),
                           locations=sol_geo.get_df(agg_filter = value).index,
                           z = sol_geo.get_df(agg_filter = value)['total_pop'],
                           colorscale="deep",
                            marker_line_width = 0,
                           #center={"lat": -8.665766, "lon": 159.012453},
                            #range_color=(0, 15000), #TODO undo hardcoding
                            zauto=True,
            visible= True if value==cols_dd[0] else False))
    
    buttons.append(dict(label=value,
                        method="update",
                        args=[{"visible":list(visible==value)},
                              {"title":f"<b>{value}</b>"}]))
updatemenus = [{"active":0,
                "buttons":buttons,
               }]

# Show figure
fig = go.Figure(data=traces,
                layout=dict(updatemenus=updatemenus))
# This is in order to get the first title displayed correctly
first_title = cols_dd[0]
fig.update_layout(title=f"<b>{first_title}</b>",
                    title_x=0.5,
                    mapbox_style = 'carto-positron',
                    mapbox_zoom = 5,
                    mapbox_center={"lat": -9.565766, "lon": 162.012453},
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
##print(fig['data'][2])


In [ ]:
# TODO the above demonstrates that we can stack mutliple graphs and toggle the visibility and potentially z value in order to 
# quickly update our graph

In [ ]:
fig = px.choropleth_mapbox(sol_geo.get_df(agg_filter = 'ward'),
        # TODO - may need to use old version of mapping to get visibility hack to work..
                        #geojson = str(repo.working_tree_dir) + "/assets/sol_geo.json",
                            geojson=sol_geo.get_geojson(agg_filter = 'ward'),
                           locations=sol_geo.get_df(agg_filter = 'ward').index,
                               color = 'total_pop',
                           color_continuous_scale="deep",
                               mapbox_style = 'carto-positron',
                               opacity = 0.8,
                        zoom = 5,
                        center={"lat": -9.565766, "lon": 162.012453})
print(fig["data"][0]["z"])

[10062  3625  2613   232  6996  1197  4343 10151  1835  1636   356  9096
 14103  1988  1574  1499  1738  1680  4931  2487  1699  1158  1746  1856
  1140  1240  1306  2397  1782   541  2514  3305  3558  4290  1967  2833
  7177  4023  5515  2238  2620  1995  4675  2477  1724  1783  2965  3610
  3050  2661  3365  2278  1929  1148  1177  1988  2813   683  1240  1862
   959  1404  2397  2163   949  1671  1907  1868  3226  2220  2026  1251
  2586  1662  2118  1722  2019  1956  2128  1520  1617   367   378   362
   239   569   117   144   265   350   250 14914  6510  3003  3118  3448
  3201  1822  1716  2262  3696  1114  3159  1477  1418  3767  4044  3068
  7438  4429 10532  4962  4515  5105  7639  6223  8830  2477  2749  5532
  4333  2760 10070  4781  1885   866  5094  4195  3592  6031  3478  3574
  3525  4988  4936  3043  2550  4650  1921  3747  5121  9634  1922  1396
   700   249  1169  1294   860  1209  3009  2009  2344  2054  3928  4562
  1625  2131  2488  3262  1547   811  1138   677  1

## Run the dash app

In [ ]:
#| export
#| server
# Run app
if __name__=='__main__':
    app.run_server(debug=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()